In [ ]:
from glob import glob
import numpy as np
import pandas as pd
pd.set_option('display.max_rows', 5)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()
%matplotlib inline
import cv2
import os, sys, time, datetime, random, math

In [ ]:
video_path = '/home/smoothjazzuser/Desktop/videogame-anomoly/MNAD/dataset/ufc/'
bugs_path = '/home/smoothjazzuser/Desktop/videogame-anomoly/MNAD/dataset/bugs/'

In [ ]:
training_path = video_path + 'training/'
testing_path = video_path + 'testing/'

# use pandas to read folder of videos
training_videos = glob(training_path + '*.mp4')
training_videos = sorted(training_videos, key=lambda x: int(x.split('Videos')[-1].split('_')[0]))
training_videos = pd.DataFrame(training_videos, columns=['video'])

testing_videos = glob(testing_path + '*//' + '*.mp4')
testing_videos = sorted(testing_videos, key=lambda x: int(x[-7:-4]))
testing_videos = pd.DataFrame(testing_videos, columns=['video'])
testing_videos['file_name'] = testing_videos['video'].apply(lambda x: x.split('/')[-1])


# parse list (by lines) of video frame labels from MNAD/dataset/ufc/Temporal_Anomaly_Annotation_for_Testing_Videos.txt
testing_labels = pd.read_csv(video_path + 'Temporal_Anomaly_Annotation_for_Testing_Videos.txt', sep=' ', header=None)
testing_videos

In [ ]:
testing_labels.columns = ['file_name', 'catagory', 'start1', 'end1', 'start2', 'end2']
#replace all -1 with None
testing_labels = testing_labels.replace(-1, None)
testing_labels.head(5)

In [ ]:
# join testing_labels to testing_videos by file_name
testing_videos = testing_videos.merge(testing_labels, on='file_name', how='left')
# drop files that are not in testing_labels
testing_videos = testing_videos.dropna(subset=['catagory'])
# convert start1 and end1 to frame index range
testing_videos['anomaly_frames'] = testing_videos.apply(lambda x: list(range(int(x['start1']), int(x['end1']))), axis=1)
del testing_videos['start1']
del testing_videos['end1']
# convert start1 and end1 to frame index range if values are not None
testing_videos['anomaly_frames2'] = testing_videos.apply(lambda x: list(range(int(x['start2']), int(x['end2']))) if not x['start2'] == None else None, axis=1)
del testing_videos['start2']
del testing_videos['end2']
#combine all intefers from anomaly_frames and anomaly_frames2
testing_videos['anomaly_frames'] = testing_videos.apply(lambda x: x['anomaly_frames'] + x['anomaly_frames2'] if not x['anomaly_frames2'] == None else x['anomaly_frames'], axis=1)
del testing_videos['anomaly_frames2']
#sort index
testing_videos = testing_videos.sort_values(by=['file_name'])
#set integer index
testing_videos = testing_videos.reset_index(drop=True)
#replace filename with integer index, and then use zfill to pad with zeros
testing_videos['file_name'] = testing_videos.index
testing_videos['file_name'] = testing_videos['file_name'].apply(lambda x: str(x).zfill(3))

testing_videos

In [ ]:
# index training
training_videos['file_name'] = training_videos.index
training_videos['file_name'] = training_videos['file_name'].apply(lambda x: str(x).zfill(3))

training_videos

In [ ]:
# calculate output folder and name for each video based on file_name
testing_videos['output_folder'] = testing_videos['file_name'].apply(lambda x: bugs_path + 'symlink_loc/frames/' + x + '/')
training_videos['output_folder'] = training_videos['file_name'].apply(lambda x: bugs_path + 'training/frames/' + x + '/')

# create output folders
testing_videos['output_folder'].apply(lambda x: os.makedirs(x, exist_ok=True))
training_videos['output_folder'].apply(lambda x: os.makedirs(x, exist_ok=True))
str(testing_videos['output_folder'][0])

In [ ]:
# use ffmpeg to load video at 30fps and save frames to output folder
def save_frames(video_path, output_folder):
    # use 95% quality to save space
    os.system('ffmpeg -i ' + video_path + ' -vf fps=30 ' + output_folder + '%3d.jpg' + ' -q:v 95')

testing_videos['video'].apply(lambda x: save_frames(x, str(testing_videos['output_folder'][testing_videos['video'] == x].values[0])))


In [ ]:

training_videos['video'].apply(lambda x: save_frames(x, str(training_videos['output_folder'][training_videos['video'] == x].values[0])))

In [ ]:
# calculate which frames are anomalies
def range_anom(output_folder, anomaly_frames):
    files = glob(output_folder + '*.jpg')
    anom = []
    for i in range(len(files)):
        if i in anomaly_frames:
            anom.append(1)
        else:
            anom.append(0)

    return anom

testing_videos['anomaly'] = testing_videos.apply(lambda x: range_anom(x['output_folder'], x['anomaly_frames']), axis=1)

In [ ]:
testing_videos

In [ ]:
#concatenate all anomaly lists into one list
frame_labels_anomalies = []
for i in range(len(testing_videos)):
    frame_labels_anomalies = frame_labels_anomalies + testing_videos['anomaly'][i]

# export the list as a .npy file called /home/smoothjazzuser/Desktop/videogame-anomoly/MNAD/dataset/bugs/symlink_loc/Gall/frame_labels_bugs.npy
np.save(bugs_path + 'symlink_loc/GTall/frame_labels_bugs.npy', np.expand_dims(frame_labels_anomalies, axis = 0))

#export to csv
testing_videos.to_csv(bugs_path + 'testing_frame_labels_bugs.csv')
training_videos.to_csv(bugs_path + 'training_frame_labels_bugs.csv')